 <h2> Paraphrase Detection with Neural Networks - Natural Language Understanding </h2>


CSI4106 Artificial Intelligence Project <br/>
Prepared by Abha Sharma & Rupsi Kaushik

<h3> Background </h3>

With the growing trends of virtual assistants and chatbots, Natural Language Processing (NLP) is a topic that is becoming increasingly popular in the recent years. From Google AI's Transformer-based models that consider a word's double-sided context to IBM's training data generator, today we have cutting edge approaches to solving NLP tasks.  However, even with these latest breakthroughs, NLP still faces many challenges, namely the problem of accurately deciphering what humans mean when they express something, regardless of how they express it. This problem falls under Natural Language Understanding (NLU), a subtopic of NLP that aims to increase the proficiency of intelligent systems in exhibiting real knowledge of natural language. Within this field, the task of paraphrase detection - determining whether a pair of sentences convey identical meaning - is considered to be an important one. Through the improvement of paraphrase detection, other NLP tasks that are integral to the efficiency of existing intelligent systems, such as question answering, information retrieval, and text summarization, can also be improved. For this reason, in this report, we propose to enhance the capability of neural networks in the context of paraphrase detection through the use of traditional Information Retrieval (IR) techniques as input features. 

<h3> Objectives </h3>

The main objective of this report is to evaluate the performance of a neural network model given different IR features. Additionally, it will take a look at how the number of features and hidden layers improve the overall performance of the model. These results will be compared among two different training sets that have been annotated for paraphrase detection. Below is the proposed architecture for our particular neural network: 

<h3> Datasets </h3>

We will be working with the Quora Question Pairs and Microsoft Research Paraphrase Corpus datasets for this project. You can find them in this folder labelled as 'msr_train.csv' and 'questions_train.csv'

In [ ]:
#Make sure to import all these modules
import pandas as pd 
from pyemd import emd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
word_vectors = api.load("glove-wiki-gigaword-100")

In [ ]:
# Taking a look at the Quora Question Pairs dataset
quora_data = pd.read_csv("questions_train.csv")
quora_data.Sentence_1 = quora_data.Sentence_1.astype(str)
quora_data.Sentence_2 = quora_data.Sentence_2.astype(str)
quora_data.head(20)

In [ ]:
#Taking a look at the Microsoft Research Paraphrase dataset 
mrp_data = pd.read_csv("msr_train.csv")
mrp_data.Sentence_1 = mrp_data.Sentence_1.astype(str)
mrp_data.Sentence_2 = mrp_data.Sentence_2.astype(str)
mrp_data.head(20)

<h3> Preprocessing </h3>

In [ ]:
def remove_stop_words(sentence):
    stop_words = stopwords.words('english')
    processed_sentence = [word for word in sentence if not word in stop_words]
    return processed_sentence 
def tokenize(sentence):
    tokenized_sentence = sentence.lower().split()
    return tokenized_sentence
def lemmatize(sentence):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = [lemmatizer.lemmatize(word) for word in sentence]
    return lemmatized_sentence

<h3>Feature Engineering</h3>

<h4> Word Mover's Distance </h4>


*Explain WMD here*

In [ ]:
def word_movers_distance(row):
    sentence_one_tokenize = tokenize(row['Sentence_1'])
    sentence_two_tokenize = tokenize(row['Sentence_2'])
    filtered_sentence_one = remove_stop_words(sentence_one_tokenize)
    filtered_sentence_two = remove_stop_words(sentence_two_tokenize)
    distance = word_vectors.wmdistance(filtered_sentence_one, filtered_sentence_two)
    return distance

In [ ]:
#Applying Word Mover's Distance to Quora sentence pairs 
quora_data['WMD_distance'] = quora_data.apply(word_movers_distance, axis=1)
quora_data.head(20)

In [ ]:
#Applying Word Mover's Distance to MRP Corpus sentence pairs 
mrp_data['WMD_distance'] = mrp_data.apply(word_movers_distance, axis=1)
mrp_data.head(20)

<h4> Pairwise Cosine Similarity </h4>

*Explain Cosine Similarity here*

In [ ]:
def calculate_cosine_similarity(row):
    sentence_one_tokenize = tokenize(row['Sentence_1'])
    sentence_two_tokenize = tokenize(row['Sentence_2'])
    filtered_sentence_one = remove_stop_words(sentence_one_tokenize)
    filtered_sentence_two = remove_stop_words(sentence_two_tokenize)
    #lemmatize to get the root words 
    lemmatize_sentence_one = lemmatize(filtered_sentence_one)
    lemmatize_sentence_two = lemmatize(filtered_sentence_two)
    tfidf = TfidfVectorizer(preprocessor=' '.join)
    tfidf_matrix = tfidf.fit_transform([lemmatize_sentence_one, lemmatize_sentence_two])
    similarity = cosine_similarity(tfidf_matrix)
    return similarity[0,1]
    

In [ ]:
#Applying Cosine Similarity to MRP Corpus sentence pairs 
mrp_data['Cosine_Similarity'] = mrp_data.apply(calculate_cosine_similarity, axis=1)
mrp_data.head(20)

<h3> Train Set & Test Set </h3>

For training with less bias, we want to maintain a reasonably equal class distribution. We want to make sure that we have a balanced class of non-paraphrase, as well as paraphrase pairs of sentences. If this is not possible, as with most real-life scenarios, we need to think of a way to adjust our metric for evaluation in order to accomodate for this imbalance. 

In [ ]:
print("Class distribution for MRP Corpus: \n{}".format(mrp_data['is_Paraphrase'].value_counts()))
print("Class distribution for Quora Corpus: \n{}".format(quora_data['is_Paraphrase'].value_counts()))

<h3> Model </h3>

<h3> Results & Evaluation </h3>

<h3> References </h3>